## Paso 2. Preprocesamiento de datos

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline as mpl
mpl.backend = "svg"
import os
import sys

Dado que en la etapa de EDA se identificaron problemas de outliers y que al quitarlos habria pérdida de datos, por ejemplo, holiday ya no se ve el valor 1, en la variable weathersit también ya no se ve data con el número 4.

Lo encontrado tambien en EDA:

- Invalid entries found in column 'yr':
- Invalid entries found in column 'mnth':
- Invalid entries found in column 'holiday':
- Invalid entries found in column 'weekday':
- Invalid entries found in column 'workingday':
- Invalid entries found in column 'weathersit':
- Invalid entries found in column 'season':

Tendremos una estrategia para restablecer los valores de las variables dependientes de la fecha o que puedan ser calculadas a partir de ella.

In [2]:
# Cargamos dataset limpio
df_mod_clean = pd.read_csv('../data/clean/bike_sharing_cleaned.csv')

In [4]:
# Revisamos el shape y los primeros registros
print(df_mod_clean.shape)
df_mod_clean.head(10)

(17726, 17)


,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,instant
0,2011-01-01,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0000,3.0,13.0,16.0,0
1,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0000,8.0,32.0,40.0,1
2,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0000,5.0,27.0,32.0,2
3,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0000,3.0,10.0,13.0,3
4,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0000,0.0,1.0,1.0,4
5,2011-01-01,1.0,0.0,1.0,5.0,0.0,6.0,0.0,2.0,0.24,0.2576,0.75,0.0896,0.0,1.0,1.0,5
6,2011-01-01,1.0,0.0,1.0,6.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0000,2.0,0.0,2.0,6
7,2011-01-01,1.0,0.0,1.0,314.0,0.0,6.0,0.0,1.0,0.20,0.2576,0.86,0.0000,1.0,2.0,3.0,7
8,2011-01-01,1.0,0.0,1.0,8.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0000,1.0,7.0,8.0,8
9,2011-01-01,1.0,NaN,1.0,9.0,0.0,6.0,0.0,1.0,0.32,0.3485,0.76,0.0000,8.0,6.0,14.0,9


In [ ]:
# Descripción estadística de las variables numéricas previo a cualquier transformación
df_mod_clean.describe().T

,count,mean,std,min,25%,50%,75%,max
season,17462.0,5.435059,40.828996,1.00,2.0000,3.0000,4.0000,985.0000
yr,17452.0,3.104343,41.013759,0.00,0.0000,1.0000,1.0000,1139.0000
mnth,17444.0,10.905354,51.858621,1.00,4.0000,7.0000,10.0000,1089.0000
hr,17433.0,17.116388,67.641675,0.00,6.0000,12.0000,18.0000,1955.0000
holiday,17477.0,2.915374,43.483761,0.00,0.0000,0.0000,0.0000,985.0000
weekday,17480.0,7.089188,96.961092,0.00,1.0000,3.0000,5.0000,11286.0000
workingday,17464.0,3.182089,39.979501,0.00,0.0000,1.0000,1.0000,971.0000
weathersit,17460.0,4.972795,46.843781,1.00,1.0000,1.0000,2.0000,989.0000
temp,17490.0,3.575313,45.033474,0.02,0.3400,0.5000,0.6600,999.4600
atemp,17474.0,3.823939,46.809246,0.00,0.3333,0.4848,0.6212,985.4545


In [ ]:
# Calculo de variables dependientes de la fecha obedeciendo las reglas de negocio del dataset original
# variables_fecha = ['yr','mnth','holiday','weekday','workingday','season']
df_mod_clean['yr'] = pd.to_datetime(df_mod_clean['dteday']).dt.year - 2011
df_mod_clean['mnth'] = pd.to_datetime(df_mod_clean['dteday']).dt.month
df_mod_clean['holiday'] = df_mod_clean['dteday'].apply(lambda x: 1 if pd.to_datetime(x).date() in [
    pd.to_datetime('2011-01-17').date(), pd.to_datetime('2011-02-21').date(), pd.to_datetime('2011-04-15').date(), pd.to_datetime('2011-05-30').date(),
    pd.to_datetime('2011-07-04').date(), pd.to_datetime('2011-09-05').date(), pd.to_datetime('2011-10-10').date(), pd.to_datetime('2011-11-11').date(),
    pd.to_datetime('2011-11-24').date(), pd.to_datetime('2011-12-26').date(), pd.to_datetime('2012-01-02').date(), pd.to_datetime('2012-01-16').date(),
    pd.to_datetime('2012-02-20').date(), pd.to_datetime('2012-04-16').date(), pd.to_datetime('2012-05-28').date(), pd.to_datetime('2012-07-04').date(),
    pd.to_datetime('2012-09-03').date(), pd.to_datetime('2012-10-08').date(), pd.to_datetime('2012-11-12').date(), pd.to_datetime('2012-11-22').date(),
    pd.to_datetime('2012-12-25').date()] else 0)
df_mod_clean['workingday'] = df_mod_clean.apply(lambda row: 1 if row['holiday'] == 0 and row['weekday'] < 5 else 0, axis=1)
df_mod_clean['weekday'] = pd.to_datetime(df_mod_clean['dteday']).dt.weekday
df_mod_clean['season'] = pd.to_datetime(df_mod_clean['dteday']).dt.month%12 // 3 + 1

In [ ]:
# Descripción estadística de las variables numéricas posterior al cálculo de las variables dependientes de la fecha
df_mod_clean.describe( include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
dteday,17531,731,2012-04-26,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
season,17531.0,NaN,NaN,NaN,2.503622,1.111022,1.0,2.0,3.0,3.0,4.0
yr,17531.0,NaN,NaN,NaN,0.502025,0.50001,0.0,0.0,1.0,1.0,1.0
mnth,17531.0,NaN,NaN,NaN,6.532428,3.43729,1.0,4.0,7.0,10.0,12.0
hr,17433.0,NaN,NaN,NaN,17.116388,67.641675,0.0,6.0,12.0,18.0,1955.0
holiday,17726.0,NaN,NaN,NaN,0.02911,0.168119,0.0,0.0,0.0,0.0,1.0
weekday,17531.0,NaN,NaN,NaN,3.004506,2.001235,0.0,1.0,3.0,5.0,6.0
workingday,17726.0,NaN,NaN,NaN,0.676295,0.467902,0.0,0.0,1.0,1.0,1.0
weathersit,17460.0,NaN,NaN,NaN,4.972795,46.843781,1.0,1.0,1.0,2.0,989.0
temp,17490.0,NaN,NaN,NaN,3.575313,45.033474,0.02,0.34,0.5,0.66,999.46


In [14]:
def numeric_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [17]:
outliers_hr = numeric_outliers(df_mod_clean, 'hr')
print(f"Number of outliers in 'hr': {len(outliers_hr)}")
outliers_weather = numeric_outliers(df_mod_clean, 'weathersit')
print(f"Number of outliers in 'weathersit': {len(outliers_weather)}")
outliers_temp = numeric_outliers(df_mod_clean, 'temp')
print(f"Number of outliers in 'temp': {len(outliers_temp)}")
outliers_atemp = numeric_outliers(df_mod_clean, 'atemp')
print(f"Number of outliers in 'atemp': {len(outliers_atemp)}")
outliers_hum = numeric_outliers(df_mod_clean, 'hum')
print(f"Number of outliers in 'hum': {len(outliers_hum)}")
outliers_wind = numeric_outliers(df_mod_clean, 'windspeed')
print(f"Number of outliers in 'windspeed': {len(outliers_wind)}")
outliers_casual = numeric_outliers(df_mod_clean, 'casual')
print(f"Number of outliers in 'casual': {len(outliers_casual)}")
outliers_registered = numeric_outliers(df_mod_clean, 'registered')
print(f"Number of outliers in 'registered': {len(outliers_registered)}")
outliers_cnt = numeric_outliers(df_mod_clean, 'cnt')
print(f"Number of outliers in 'cnt': {len(outliers_cnt)}")

Number of outliers in 'hr': 173
Number of outliers in 'weathersit': 183
Number of outliers in 'temp': 185
Number of outliers in 'atemp': 196
Number of outliers in 'hum': 190
Number of outliers in 'windspeed': 267
Number of outliers in 'casual': 1280
Number of outliers in 'registered': 775
Number of outliers in 'cnt': 586


In [16]:
outliers_hr.describe().T

,count,mean,std,min,25%,50%,75%,max
season,172.0,2.552326,1.104295,1.0000,2.0000,2.5000,4.0000,4.0000
yr,172.0,0.511628,0.501324,0.0000,0.0000,1.0000,1.0000,1.0000
mnth,172.0,6.476744,3.432649,1.0000,3.0000,6.0000,10.0000,12.0000
hr,173.0,570.439306,384.555033,38.0000,262.0000,517.0000,789.0000,1955.0000
holiday,173.0,0.023121,0.150725,0.0000,0.0000,0.0000,0.0000,1.0000
weekday,172.0,2.988372,2.066124,0.0000,1.0000,3.0000,5.0000,6.0000
workingday,173.0,0.687861,0.464711,0.0000,0.0000,1.0000,1.0000,1.0000
weathersit,171.0,4.362573,28.618538,1.0000,1.0000,1.0000,2.0000,344.0000
temp,170.0,7.264588,67.567088,0.0600,0.3400,0.5100,0.6600,818.1600
atemp,172.0,5.369013,64.258768,0.0606,0.3144,0.4848,0.6212,843.2121


In [ ]:
numeric_cols = df_mod_clean.select_dtypes(include=['number', 'float']).columns
# Dado que la variable holiday es binaria, no se aplica el tratamiento de outliers clasico por IQR
numeric_cols = numeric_cols.drop('holiday') 
# Tratamiento de outliers usando el método del IQR
for col in numeric_cols:
    # Calculate IQR
    Q1 = df_mod_clean[col].quantile(0.25)
    Q3 = df_mod_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    # Define outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Remove outliers
    df_mod_clean = df_mod_clean[(df_mod_clean[col] >= lower_bound) & (df_mod_clean[col] <= upper_bound)]

In [ ]:
# Búsqueda y tratamiento de outliers en df_mod
numeric_cols = df_mod_clean.select_dtypes(include=['number', 'float']).columns
for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df_mod_clean[col])
    plt.title(f'Boxplot of {col}')
    plt.show()

In [ ]:
# Revisando el efecto del tratamiento de outliers en las variables numéricas
df_mod_clean.describe(include=['number', 'float']).T
# Falta procesar la variable de holiday

Vemos que se cumple, siempre y cuando el weekday sea diferente de 0 o 6, lo cual supone fin de semana. Por lo que podemos hacer la imputación de holiday de la siguiente forma:
- holiday = 1, cuando workingday = 0 y weekday no es 0 ni 6
- holiday = 0, cuando workingday = 1

In [ ]:
def impute_holiday(df):
    """
    Imputa valores de holiday basado en workingday y weekday:
    - holiday = 1, cuando workingday = 0 y weekday no es 0 ni 6
    - holiday = 0, cuando workingday = 1
    """
    df_copy = df.copy()
    
    # Condición 1: holiday = 0 cuando workingday = 1
    mask_workingday_1 = df_copy['workingday'] == 1
    df_copy.loc[mask_workingday_1, 'holiday'] = 0
    
    # Condición 2: holiday = 1 cuando workingday = 0 y weekday no es 0 ni 6
    mask_workingday_0 = (df_copy['workingday'] == 0) & (~df_copy['weekday'].isin([0, 6]))
    df_copy.loc[mask_workingday_0, 'holiday'] = 1
    
    # Condicion 3: holiday = 0 cuando workingday = 0 y weekday es 0 o 6
    mask_workingday_0_weekend = (df_copy['workingday'] == 0) & (df_copy['weekday'].isin([0, 6]))
    df_copy.loc[mask_workingday_0_weekend, 'holiday'] = 0

    return df_copy

# Aplicar la función de imputación
df_mod_clean = impute_holiday(df_mod_clean)

In [ ]:
def plot_distribution(df_mod_clean, column):
    plt.figure(figsize=(10, 6))
    sns.histplot(df_mod_clean[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

for col in df_mod_clean.select_dtypes(include=['number', 'float']).columns:
    plot_distribution(df_mod_clean, col) 